# Final Exam: Temperature Conditions on Bikeways in Bavaria

Author: Agnes Zwick
Lecturer: Steven Hill




<div style="display: flex;">
    <img src="https://www.ardalpha.de/wissen/umwelt/klima/klimawandel/temperaturen-bayern-sommer-2018-klimawandel-100~_v-img__16__9__xl_-d31c35f8186ebeb80b0cd843a7c267a0e0c81647.png?version=9e469" alt="Image 1" style="max-height: 200px; margin-right: 20px;">
    <img src="https://www.radlland-bayern.de/wp-content/uploads/img-rv-1080x720-knotenpunkt.jpg" alt="Image 2" style="height: 200px; width: auto; object-fit: cover; object-position: 100% 0;">
</div>


The primary objective of this project is to identify and analyze the coldest temperature zones along bikeways connecting cities within Bavaria. The focus will be on determining bikeways that experience lower temperatures, contributing to a comprehensive understanding of temperature conditions for cyclists.

## 1. Vector Data Analysis
The code's purpose is to prepare vector data for later use in a pathfinding process.

**- Segmentation of Bikeways:**
The code proceeds to segment the bikeways into distinct individual sections. This is achieved by introducing intersection points at specific locations along the bikeways. By doing so, the bikeways are divided into segments that can be effectively navigated during the pathfinding process.

## 2. Raster Data Analysis
The temperature information regarding the bikeways is provided through raster data sourced from MODIS. This data is then subject to the following steps:

**- Reprojection and Clipping:**
The initial action involves the reprojection of the temperature raster data to align with the geographic coordinate system of Bavaria. Additionally, the data is clipped to match the boundary of Bavaria, ensuring that only relevant information is retained.

**- Zonal Statistics Calculation:**
Zonal statistics are computed for each individual bikeway. This process entails analyzing the temperature data within the spatial boundaries of each bikeway.

## 4. Final Map
To generate an informative map, the following steps are executed:

**- Centroid Point Creation:**
Points are created at the centroids of each municipal area. These points serve as representative markers for each municipality.

**- Population Data Integration:**
A CSV file containing population data for Bavarian municipalities is loaded. The integration of this data provides a more comprehensive overview of the area of interest.

**- Population-Based Filtering:**
The generated points of each municipal are filtered based on population values.

By following these steps, the map effectively visualizes the Bavarian municipalities, highlighting those with significant populations. This aids in improving the spatial understanding of the pathway within Bavaria.






# Start and End Point Definition

The initial step involves defining the start and end points for the pathfinding process. For this purpose, a municipality within Bavaria is selected as the starting point. Subsequently, a random point is generated on a bikeway within this chosen municipality. These defined points establish the journey's origin and destination, initiating the pathfinding analysis.

In [ ]:
start = "Regensburg"
end = "München"

## 1. Vector Data Analysis
To initiate the vector data analysis, it's essential to import the necessary libraries.

In [ ]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point, LineString, MultiLineString

**Get/Set working directory**

In [ ]:
os.getcwd()

**Import data**

In [ ]:
kataster_bavaria = gpd.read_file('../Final_Exam/input_data/vector/KatasterBezirk.shp')
bikeways = gpd.read_file('../Final_Exam/input_data/vector/bikeways.shp')

**Vector Data Tasks:**

- Generate a polygon outlining the Bavarian borders utilizing the KatasterBezirk.shp dataset.
- Import the bikeways.shp dataset.
  - Convert MultiLineString geometries to LineString.
  - Establish intersection points where two or more bikeways intersect.
  - Manually eliminate extraneous points that may have been falsely created on lines using QGIS.
  - Segment the bikeways into individual lines by utilizing the intersection points.
  - Remove duplicates present among the intersection points and individual line segments.

**Dissolving kataster_bavaria into Bavarian Outlines**

In [ ]:
bavaria = kataster_bavaria.dissolve()

**Convert MultiLineString geometries to LineString**


In [ ]:
multiline = bikeways.geometry.unary_union
if isinstance(multiline, LineString):
    multiline = MultiLineString([multiline])

# Create a new GeoDataFrame with a single MultiLineString
modified_bikeways = gpd.GeoDataFrame(geometry=[multiline], crs=bikeways.crs)

# Save the modified shapefile
modified_bikeways.to_file("../Final_Exam/output_data/vector/modified_bikeways.shp", driver='ESRI Shapefile')

**Generation of Intersection Points**

In [ ]:
# Extract the MultiLineString from the modified shapefile
multiline = modified_bikeways.geometry[0]

# Create a list to store the start and end points
points = []

# Iterate through each line segment in the MultiLineString
for line_segment in multiline.geoms:
    points.extend([Point(line_segment.coords[0]), Point(line_segment.coords[-1])])

# Add points at the start and end of each bikeway
for line in multiline.geoms:
    points.extend([Point(line.coords[0]), Point(line.coords[-1])])

# Convert the list of points to a GeoDataFrame
points_gdf = gpd.GeoDataFrame(geometry=points, crs=modified_bikeways.crs)

# Save the points as a new shapefile
points_gdf.to_file("../Final_Exam/output_data/vector/intersections.shp", driver='ESRI Shapefile')

**Manual Removal of Non-Intersection Points in QGIS**

In [ ]:
intersections = gpd.read_file("../Final_Exam/input_data/vector/intersections_mod.shp")

**Remove duplicates of intersections**


In [ ]:
intersections = intersections.drop_duplicates(subset='geometry')
print(intersections)

**Segmentation of Bikeways into Individual Lines Using Intersection Points**

In [ ]:
paths = []

for line in bikeways.geometry:
    if line.geom_type == 'LineString':
        line_coords = list(line.coords)
        for point in intersections.geometry:
            if point.distance(line) < 1e-5 and line.distance(point) < 1e-5: 
                if point.coords[0] in line_coords:
                    # Split the line based on the intersection point
                    index = line_coords.index(point.coords[0])
                    if index > 0:
                        split_lines = LineString(line_coords[:index + 1])
                        paths.append(split_lines)

# Create a new GeoDataFrame for the individual path segments and set CRS
indivual_bikeways = gpd.GeoDataFrame(geometry=paths, crs=bikeways.crs)

# Save the individual path segments as a shapefile
indivual_bikeways.to_file('../Final_Exam/output_data/vector/indivual_bikeways.shp')

**Elimination of Duplicate Line Segments**

In [ ]:
indivual_bikeways = indivual_bikeways.drop_duplicates(subset='geometry')
print(indivual_bikeways)

## 2. Raster Data Analysis
To initiate the raster data analysis, it's essential to import the necessary libraries.

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.plot import show
from rasterio.mask import mask
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

**Raster Data Tasks:**

- Import the raster file containing temperature data for the summer month.
- Reproject the raster file to align with the project's coordinate system.
- Clip and mask the raster data to match the outline of Bavaria's geographical boundaries.

In [ ]:
# Open the source raster
src_file = "../Final_Exam/input_data/raster/mean_LST_summer.tif"
srcRst = rasterio.open(src_file)

# Define the target CRS (EPSG:25832)
dst_crs = bikeways.crs

# Calculate the transform array and shape of the reprojected raster
transform, width, height = calculate_default_transform(
    srcRst.crs, dst_crs, srcRst.width, srcRst.height, *srcRst.bounds)

# Prepare the metadata for the destination raster
kwargs = srcRst.meta.copy()
kwargs.update({
    'crs': dst_crs,
    'transform': transform,
    'width': width,
    'height': height
})

# Open the destination raster in write mode
dst_file = "../Final_Exam/output_data/raster/reproj_mean_LST_summer.tif"
dstRst = rasterio.open(dst_file, 'w', **kwargs)

# Reproject and save the raster band data
for i in range(1, srcRst.count + 1):
    reproject(
        source=rasterio.band(srcRst, i),
        destination=rasterio.band(dstRst, i),
        src_transform=srcRst.transform,
        src_crs=srcRst.crs,
        dst_transform=transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest)

# Close the destination raster
dstRst.close()

In [ ]:
LST = "../Final_Exam/output_data/raster/reproj_mean_LST_summer.tif"

**Clipping and Masking Raster Data**

In [ ]:
# Open the GeoTIFF file
srcRst = rasterio.open(LST)

# Clip the raster to the extent of bavaria
clipped, out_transform = mask(srcRst, shapes=bavaria.geometry, crop=True)

# Set values outside the shapefile boundary to NA
clipped = np.where(clipped == srcRst.nodata, np.nan, clipped)

# Update metadata for the clipped raster
clipped_meta = srcRst.meta.copy()
clipped_meta.update({
    "height": clipped.shape[1],
    "width": clipped.shape[2],
    "transform": out_transform,
    "dtype": 'float32',  # Set dtype as float32 to handle NaN values
    "nodata": np.nan  # Set nodata value to NaN
})

# Plot the clipped and masked raster
plt.figure(figsize=(10, 10))  # Set the figure size (adjust as needed)
plt.imshow(clipped[0], cmap='viridis')  # Use the 'viridis' colormap, but you can choose any other colormap
plt.title("Clipped and Masked LST of the summer month 2022 in Bavaria")  # Add a title to the plot (optional)
plt.colorbar(label='Temperature (°C)')  # Add a colorbar with the label (optional)
plt.show()

# Close the GeoTIFF file after clipping
srcRst.close()

# Specify the path and filename for the clipped raster
clipped_file = '../Final_Exam/output_data/raster/clipped_reproj_mean_LST.tif'

# Save the clipped raster as a new GeoTIFF file
with rasterio.open(clipped_file, 'w', **clipped_meta) as dst:
    dst.write(clipped)

In [ ]:
LST = "../Final_Exam/output_data/raster/clipped_reproj_mean_LST.tif"

## 3. Pathfinder
To initiate the pathfinder, it's essential to import the necessary libraries.

In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
import rasterstats
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import nearest_points
import random
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import numpy as np
import csv
from adjustText import adjust_text


**Pathfinder Tasks:**

- Compute the mean temperature for each individual bikeway.
- Construct a network graph for pathfinding purposes.
  - Define the start and end points of the "trip" based on information from the kataster file.
  - Determine the coldest and shortest path from the start to the end point.
- Calculate the length and mean temperature of the chosen path.

**Calculation of Mean Temperature for Each Individual Bikeway**

In [ ]:
# Load split line shapefile and MODIS LST raster
LST = rasterio.open(LST)
temperature_values = LST.read(1)

# Create an empty list to store mean temperatures
mean_temperatures = []

# Iterate over split lines
for index, row in indivual_bikeways.iterrows():
    line = row['geometry']
    
    # Extract temperature values using zonal statistics
    line_stats = rasterstats.zonal_stats(line, temperature_values, affine=LST.transform, stats='mean')
    
    # Get the mean temperature from the statistics
    if line_stats and line_stats[0]['mean'] is not None:
        mean_temp = line_stats[0]['mean']
        mean_temperatures.append(mean_temp)
    else:
        mean_temperatures.append(None)

# Add mean temperature values to the split lines GeoDataFrame
indivual_bikeways['mean_temp'] = mean_temperatures
indivual_bikeways

**Generation of Random Point within Start and End Municipality Polygons on a Bikeway**

In [ ]:
# Select the polygons with the same municipal name
start_town = kataster_bavaria[kataster_bavaria["gemeinde"] == start]
end_town = kataster_bavaria[kataster_bavaria["gemeinde"] == end]

# Merge the selected polygons into a single polygon
start_town = start_town.dissolve()
end_town = end_town.dissolve()

start_bikeways = gpd.clip(indivual_bikeways, start_town)
end_bikeways = gpd.clip(indivual_bikeways, end_town)

# Generate a random sample of LineString objects from the clipped bikeways
random_sample_start = start_bikeways.sample(n=1)
random_sample_end = end_bikeways.sample(n=1)

# Get a random distance along the LineString
line_length_start = random_sample_start.geometry.length.values[0]
random_distance_start = random.uniform(0, line_length_start)

line_length_end = random_sample_end.geometry.length.values[0]
random_distance_end = random.uniform(0, line_length_end)

# Generate a random point on the LineString at the chosen distance
random_point_start = random_sample_start.geometry.interpolate(random_distance_start)
random_point_end = random_sample_end.geometry.interpolate(random_distance_end)

print(random_point_start, random_point_end)

**Pathfinding Process**

In [ ]:
# Create an empty graph
G = nx.Graph()

# Iterate over each line feature in the shapefile
for i, line in indivual_bikeways.iterrows():
    line_geometry = line.geometry  # Get the line's geometry

    # Get the mean temperature from the attribute (assuming 'mean_temp' as the attribute name)
    mean_temperature = line['mean_temp']

    # Extract the coordinates from the line's geometry
    line_coords = list(line_geometry.coords)

    # Iterate over each pair of consecutive coordinates and add edges to the graph
    for j in range(len(line_coords) - 1):
        start_point = line_coords[j]
        end_point = line_coords[j + 1]
        G.add_edge(start_point, end_point, weight=mean_temperature)

# Assign start and end points from the previous code
start_point = (random_point_start.geometry.values[0].x, random_point_start.geometry.values[0].y)
end_point = (random_point_end.geometry.values[0].x, random_point_end.geometry.values[0].y)

# Calculate the distances between the start and end points and the nodes in the graph
start_distances = {node: ((node[0] - start_point[0]) ** 2 + (node[1] - start_point[1]) ** 2) ** 0.5 for node in G.nodes()}
end_distances = {node: ((node[0] - end_point[0]) ** 2 + (node[1] - end_point[1]) ** 2) ** 0.5 for node in G.nodes()}

# Find the nearest nodes in the graph to the start and end points
start_node = min(start_distances, key=start_distances.get)
end_node = min(end_distances, key=end_distances.get)

# Find the coldest and shortest path using the shortest_path function
coldest_path = nx.shortest_path(G, source=start_node, target=end_node, weight='weight')

# Convert coldest_path to LineString
coldest_path_line = LineString(coldest_path)

# Convert start and end points to Point objects
start_point_geom = Point(start_point)
end_point_geom = Point(end_point)

**Calculation of Length and Mean Temperature for the Selected Bikeway**

In [ ]:
# Calculate the length of the coldest path line in kilometers
coldest_path_length_km = coldest_path_line.length / 1000
print(f"Length of the coldest path between {start} and {end}:", 
      round(coldest_path_length_km, 2),"km")

In [ ]:
# Open the raster file
LST = rasterio.open("../Final_Exam/output_data/raster/clipped_reproj_mean_LST.tif")

# Initialize an empty list to store the temperature values
temperatures = []

# Iterate over each point along the coldest_path_line
for point in coldest_path_line.coords:
    # Get the pixel value from the raster corresponding to the point
    row, col = LST.index(point[0], point[1])
    temperature = LST.read(1, window=((row, row+1), (col, col+1)))
    temperatures.append(temperature)

# Calculate the mean temperature of the coldest_path_line
mean_temperature = np.mean(temperatures)

# Print the mean temperature of the coldest_path_line
print(f"Mean temperature on the path between {start} and {end}:", 
      round(float(mean_temperature), 2), "°C")

## 4. Final Map
The final map is composed of:

- Centroid Points: Strategically positioned at municipal centroids.
- Population Integration: CSV population data integrated for demographic context.
- Population Filtering: Points filtered based on population
- Coldest path between start and end
- Result: Overview Map and Detailed Map with information of the length of each bikeway in Bavaria

In [ ]:
# Dissolve the polygons based on the "gemeinde" column
municipals_poly = kataster_bavaria.dissolve(by='gemeinde')

# Create a new column to store the "gemeinde" name
municipals_poly['municipal_name'] = municipals_poly.index

# Create a new column to store the centroid points
municipals_poly['centroid'] = municipals_poly['geometry'].centroid

# # Convert the GeoDataFrame to a GeoSeries of Point objects
municipals_centroids = municipals_poly['centroid'].apply(lambda p: Point(p.x, p.y))

# # Create a new GeoDataFrame for the points
municipals_centroids = gpd.GeoDataFrame(geometry=municipals_centroids)

# Assign the "point_name" column to the new GeoDataFrame
municipals_centroids['municipal_name'] = municipals_poly['municipal_name']

In [ ]:
# Specify the path to your CSV file
csv_file_path = "../Final_Exam/input_data/table/population_bavaria.csv"

# Read the CSV file and store the "name" and "einwohner_2021" columns in a dictionary
einwohner_data = {}
with open(csv_file_path, "r", encoding="utf-8-sig") as file:
    csv_reader = csv.reader(file, delimiter=";")
    header_row = next(csv_reader)  # Store the header row
    for row in csv_reader:
        name = row[0]
        einwohner_2021 = row[6]
        einwohner_data[name] = einwohner_2021

# Remove the first row from einwohner_data dictionary
einwohner_data.pop('')

# Remove the "." character from 'einwohner_2021' values in einwohner_data dictionary
einwohner_data = {key: value.replace('.', '') for key, value in einwohner_data.items()}

# Create a new column 'einwohner_2021' in municipals_centroids and initialize it with NaN
municipals_centroids['einwohner_2021'] = float('NaN')

# Map 'einwohner_2021' values from einwohner_data dictionary to municipals_centroids
for name in municipals_centroids['municipal_name']:
    names = name.split(';')
    for individual_name in names:
        if individual_name in einwohner_data:
            mask = municipals_centroids['municipal_name'] == name
            municipals_centroids.loc[mask, 'einwohner_2021'] = einwohner_data[individual_name]
            break
    else:
        print(f"No population data found for '{name}'")

# Remove points where 'einwohner_2021' is still NaN
municipals_centroids_population = municipals_centroids[~municipals_centroids['einwohner_2021'].isna()].copy()

# Convert 'einwohner_2021' column to numeric values
municipals_centroids_population['einwohner_2021'] = pd.to_numeric(municipals_centroids_population['einwohner_2021'], errors='coerce')


In [ ]:
# Remove rows with 'einwohner_2021' below 50000
municipals_gt_100k = municipals_centroids_population[municipals_centroids_population['einwohner_2021'] >= 100000]
municipals_gt_100k

# Create a dictionary to store unique names and their corresponding population and geometry
unique_names_data2 = {}

# Iterate through the rows and add the population and geometry to the dictionary
for idx, row in municipals_gt_100k.iterrows():
    names = row['municipal_name'].split(';')
    einwohner_2021 = row['einwohner_2021']
    geometry = row['geometry']
    
    for name in names:
        if name not in unique_names_data2:
            unique_names_data2[name] = {'population': einwohner_2021, 'geometry': geometry}
            
# Create a GeoDataFrame with the unique names, population, and geometry

unique_names_gdf2 = gpd.GeoDataFrame(columns=['municipal_name', 'einwohner_2021', 'geometry'])
for name, data in unique_names_data2.items():
    unique_names_gdf2 = unique_names_gdf2.append({'municipal_name': name, 'einwohner_2021': data['population'], 'geometry': data['geometry']}, ignore_index=True)


In [ ]:
coldest_path_gdf = gpd.GeoDataFrame(geometry=[coldest_path_line])
start_gdf = gpd.GeoDataFrame(geometry=[start_point_geom])
end_gdf = gpd.GeoDataFrame(geometry=[end_point_geom])

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from adjustText import adjust_text

# Assuming you have the necessary GeoDataFrames and LineString

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Plot Bavaria outline
bavaria.plot(ax=ax, color='lightgray', edgecolor='black')

# Plot the coldest path
coldest_path_gdf.plot(ax=ax, color='blue', linewidth=2, label='Coldest Path')

bikeways.plot(ax=ax, color='black', alpha=0.3, linewidth=1, label='Bikeways')

# Plot the start and end points
start_gdf.plot(ax=ax, color='green', markersize=100, label='Start Point')
end_gdf.plot(ax=ax, color='red', markersize=100, label='End Point')

# Plot the unique names and their population on the map
points = unique_names_gdf2.plot(ax=ax, color='black', markersize=10, label='Cities with a population >100k', alpha=0.5)

# Create a list of texts to label the points
texts = []
for idx, row in unique_names_gdf2.iterrows():
    texts.append(ax.text(row.geometry.x, row.geometry.y, row.municipal_name, fontsize=8, ha='center', va='center'))

# Adjust the positions of the labels to avoid overlap
adjust_text(texts)

# Set plot title and labels
plt.legend()

# Set the dynamic plot title
title = f"Overview of Coldest Bikeways from {start} to {end}"

# Set plot title and axis labels
plt.title(title)
plt.xlabel("Longitude")
plt.ylabel("Latitude")

# Show the plot
plt.show()


In [ ]:
# Remove rows with 'einwohner_2021' below 10000
municipals_gt_10k = municipals_centroids_population[municipals_centroids_population['einwohner_2021'] >= 10000]
municipals_gt_10k

# Create a dictionary to store unique names and their corresponding population and geometry
unique_names_data = {}

# Iterate through the rows and add the population and geometry to the dictionary
for idx, row in municipals_gt_10k.iterrows():
    names = row['municipal_name'].split(';')
    einwohner_2021 = row['einwohner_2021']
    geometry = row['geometry']
    
    for name in names:
        if name not in unique_names_data:
            unique_names_data[name] = {'population': einwohner_2021, 'geometry': geometry}
            
# Create a GeoDataFrame with the unique names, population, and geometry

unique_names_gdf = gpd.GeoDataFrame(columns=['municipal_name', 'einwohner_2021', 'geometry'])
for name, data in unique_names_data.items():
    unique_names_gdf = unique_names_gdf.append({'municipal_name': name, 'einwohner_2021': data['population'], 'geometry': data['geometry']}, ignore_index=True)


In [ ]:
# Convert the coldest_path_line to a polygon by buffering
clip_mask = coldest_path_line.buffer(0.001)  # Adjust the buffer distance if needed

# Clip the bikeway data to the extent of the coldest line
coldest_bikeway = gpd.clip(bikeways, clip_mask)

# Convert MultiLineString to LineString
coldest_bikeway = coldest_bikeway.explode()

# Calculate and add the length (in km) in a new column
coldest_bikeway['length'] = coldest_bikeway.geometry.length / 1000

# Filter the bikeways longer than 1 km
coldest_bikeway_1km = coldest_bikeway[coldest_bikeway['length'] > 1]

# Plot the bikeways longer than 1 km
fig, ax = plt.subplots(figsize=(15, 15))

# Create a red gradient color map for the line colors
lengths = coldest_bikeway_1km['length']
cmap = plt.cm.get_cmap('Reds')
norm = plt.Normalize(vmin=lengths.min(), vmax=lengths.max())
scalar_map = ScalarMappable(cmap=cmap, norm=norm)

texts = []  # List to store the text objects

for idx, row in coldest_bikeway_1km.iterrows():
    bikeway_name = row['Name']
    length = row['length']

    # Assign a unique color based on the length using the color map
    line_color = scalar_map.to_rgba(length)

    # Extract line coordinates
    line_x, line_y = row.geometry.xy

    ax.plot(line_x, line_y, color=line_color, linewidth=2)

    # Calculate the centroid of the line geometry
    centroid_x, centroid_y = row.geometry.centroid.xy

    # Add the bikeway name as a label to the centroid of the line
    texts.append(
        ax.text(
            centroid_x[0],
            centroid_y[0],
            f"{bikeway_name} ({length:.2f} km)",
            fontsize=10,
            ha='center',
            va='center'
        )
    )

# Municipals with a population greater than 5000 within the extent of the map
unique_names_gdf = unique_names_gdf.cx[
    ax.get_xlim()[0]:ax.get_xlim()[1],
    ax.get_ylim()[0]:ax.get_ylim()[1]
]

# Add the start and end municipal polygons
clipped_start_town = start_town.cx[
    ax.get_xlim()[0]:ax.get_xlim()[1],
    ax.get_ylim()[0]:ax.get_ylim()[1]
]
clipped_end_town = end_town.cx[
    ax.get_xlim()[0]:ax.get_xlim()[1],
    ax.get_ylim()[0]:ax.get_ylim()[1]
]

start_point = gpd.GeoDataFrame(geometry=[Point(random_point_start.geometry.values[0].x, random_point_start.geometry.values[0].y)])
end_point = gpd.GeoDataFrame(geometry=[Point(random_point_end.geometry.values[0].x, random_point_end.geometry.values[0].y)])

# Set the coordinate system of the start and end points to match the map
start_point = start_point.set_crs(bikeways.crs)
end_point = end_point.set_crs(bikeways.crs)

# Plot the start and end points on the map
ax.scatter(start_point.geometry.x, start_point.geometry.y, color='red', label=start, s=10)
ax.scatter(end_point.geometry.x, end_point.geometry.y, color='red', label=end, s=10)

# Add labels for the start and end points
ax.text(start_point.geometry.x, start_point.geometry.y, start, fontsize=8, va='bottom', ha='center')
ax.text(end_point.geometry.x, end_point.geometry.y, end, fontsize=8, va='bottom', ha='center')

# Plot the outlines of start and end municipals
clipped_start_town.boundary.plot(ax=ax, color='grey', linewidth=0.5)
clipped_end_town.boundary.plot(ax=ax, color='grey', linewidth=0.5)

# Plot the municipals_gt_5000 points on the map
ax.scatter(
    unique_names_gdf.geometry.x,
    unique_names_gdf.geometry.y,
    color='grey',
    label='municipal_name',
    s=10
)


# Add labels to the filtered points
for x, y, label in zip(
    unique_names_gdf.geometry.x,
    unique_names_gdf.geometry.y,
    unique_names_gdf['municipal_name']
):
    texts.append(
        ax.text(
            x,
            y,
            label,
            fontsize=8,
            ha='center',
            va='center',
            color='grey'
        )
    )

# Automatically adjust label positions to prevent overlap
adjust_text(texts, ax=ax)

# Create the colorbar for the length gradient
cbar = plt.colorbar(scalar_map, shrink=0.2)
cbar.set_label('Length (km)')

# Set the dynamic plot title
title = f"Coldest Bikeways from {start} to {end}"

# Set plot title and axis labels
plt.title(title)
plt.xlabel("Longitude")
plt.ylabel("Latitude")

# Show the plot
plt.show()